# Logging Experiments

``rubicon_ml``'s core functionality is centered around logging **experiments** to explain and explore various
model runs throughout the model development lifecycle. This example will take a quick look at how we can log
model metadata to ``rubicon_ml`` in the context of a simple classification project.

We'll leverage the ``palmerpenguins`` dataset collected by Dr. Kristen Gorman as our training/testing data. More
information on the dataset can be found here:
> https://allisonhorst.github.io/palmerpenguins/

In [1]:
! pip install palmerpenguins

In [2]:
from palmerpenguins import load_penguins

penguins_df = load_penguins()
target_values = penguins_df['species'].unique()

print(f"target classes (species): {target_values}")
penguins_df.head()

target classes (species): ['Adelie' 'Gentoo' 'Chinstrap']


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [3]:
from sklearn.preprocessing import LabelEncoder

for column in ["species", "island", "sex"]:
    penguins_df[column] = LabelEncoder().fit_transform(penguins_df[column])

print(f"target classes (species): {penguins_df['species'].unique()}")
penguins_df.head()

target classes (species): [0 2 1]


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,0,2,39.1,18.7,181.0,3750.0,1,2007
1,0,2,39.5,17.4,186.0,3800.0,0,2007
2,0,2,40.3,18.0,195.0,3250.0,0,2007
3,0,2,NaN,NaN,NaN,NaN,2,2007
4,0,2,36.7,19.3,193.0,3450.0,0,2007


In [4]:
from sklearn.model_selection import train_test_split

train_penguins_df, test_penguins_df = train_test_split(penguins_df, test_size=.30)

target_name = "species"
feature_names = [c for c in train_penguins_df.columns if c != target_name]

X_train, y_train = train_penguins_df[feature_names], train_penguins_df[target_name]
X_test, y_test = test_penguins_df[feature_names], test_penguins_df[target_name]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((240, 7), (240,), (104, 7), (104,))

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

imputer_strategy = "mean"
classifier_n_neighbors = 5

steps = [
    ("si", SimpleImputer(strategy=imputer_strategy)),
    ("kn", KNeighborsClassifier(n_neighbors=classifier_n_neighbors)),
]

penguin_pipeline = Pipeline(steps=steps)
penguin_pipeline.fit(X_train, y_train)

score = penguin_pipeline.score(X_test, y_test)
score

0.8365384615384616

In [6]:
from rubicon_ml import Rubicon

rubicon = Rubicon(
    persistence="filesystem",
    root_dir="./rubicon-root",
    auto_git_enabled=True,
)
project = rubicon.get_or_create_project(name="classifying penguins")
experiment = project.log_experiment()

for feature_name in feature_names:
    experiment.log_feature(name=feature_name)

experiment.log_parameter(name="strategy", value=imputer_strategy)
experiment.log_parameter(name="n_neighbors", value=classifier_n_neighbors)
experiment.log_metric(name="accuracy", value=score)

print(experiment)
print()
print(f"git info:")
print(f"\tbranch name: {experiment.branch_name}\n\tcommit hash: {experiment.commit_hash}")
print(f"features: {[f.name for f in experiment.features()]}")
print(f"parameters: {[(p.name, p.value) for p in experiment.parameters()]}")
print(f"metrics: {[(m.name, m.value) for m in experiment.metrics()]}")

Experiment(project_name='classifying penguins', id='ef60d24a-429c-440f-a5b6-b5ef60f1b0de', name=None, description=None, model_name=None, branch_name='210-new-quick-look', commit_hash='9db3754e91b36ad9c9bfa55a477daf2459bf6ea4', training_metadata=None, tags=[], created_at=datetime.datetime(2022, 6, 21, 16, 48, 14, 12370))

git info:
	branch name: 210-new-quick-look
	commit hash: 9db3754e91b36ad9c9bfa55a477daf2459bf6ea4
features: ['island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex', 'year']
parameters: [('strategy', 'mean'), ('n_neighbors', 5)]
metrics: [('accuracy', 0.8365384615384616)]


In [7]:
from sklearn.base import clone

for imputer_strategy in ["mean", "median", "most_frequent"]:
    for classifier_n_neighbors in [5, 10, 15, 20]:
        pipeline = clone(penguin_pipeline)
        pipeline.set_params(
            si__strategy=imputer_strategy,
            kn__n_neighbors=classifier_n_neighbors,
        )
        
        pipeline.fit(X_train, y_train)
        score = pipeline.score(X_test, y_test)

        experiment = project.log_experiment(tags=["parameter search"])

        for feature_name in feature_names:
            experiment.log_feature(name=feature_name)
        experiment.log_parameter(name="strategy", value=imputer_strategy)
        experiment.log_parameter(name="n_neighbors", value=classifier_n_neighbors)
        experiment.log_metric(name="accuracy", value=score)

print("experiments:")
for experiment in project.experiments(tags=["parameter search"]):
    print(
        f"\tid: {experiment.id}, "
        f"parameters: {[(p.name, p.value) for p in experiment.parameters()]}, "
        f"metrics: {[(m.name, m.value) for m in experiment.metrics()]}"
    )

experiments:
	id: 76c9d99f-f398-4f25-b4c0-99d81377f026, parameters: [('strategy', 'mean'), ('n_neighbors', 5)], metrics: [('accuracy', 0.8365384615384616)]
	id: 45c5a25f-2120-438c-a947-38f512b9c861, parameters: [('strategy', 'mean'), ('n_neighbors', 10)], metrics: [('accuracy', 0.7788461538461539)]
	id: 61c8e495-aea8-4160-af39-82038326bc24, parameters: [('strategy', 'mean'), ('n_neighbors', 15)], metrics: [('accuracy', 0.75)]
	id: a9791635-27c7-4eb4-9bda-9e03533134a7, parameters: [('strategy', 'mean'), ('n_neighbors', 20)], metrics: [('accuracy', 0.7596153846153846)]
	id: 1d3ec0ec-4013-46bb-88b7-598db3816c16, parameters: [('strategy', 'median'), ('n_neighbors', 5)], metrics: [('accuracy', 0.8269230769230769)]
	id: f3c62c55-349b-467f-9eb1-4756bbe36409, parameters: [('strategy', 'median'), ('n_neighbors', 10)], metrics: [('accuracy', 0.7788461538461539)]
	id: 44963bd2-5e13-4d2c-8b64-cf26017a6871, parameters: [('strategy', 'median'), ('n_neighbors', 15)], metrics: [('accuracy', 0.75)]
	id

In [8]:
import pickle
import pandas as pd
from sklearn.metrics import confusion_matrix

experiment = project.experiments(tags=["parameter search"])[-1]

trained_model = pipeline._final_estimator
experiment.log_artifact(data_bytes=pickle.dumps(trained_model), name="trained model")

y_pred = pipeline.predict(X_test)
confusion_matrix_df = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=target_values,
    index=target_values,
)
experiment.log_dataframe(confusion_matrix_df, name="confusion matrix")

print(pickle.loads(experiment.artifact(name="trained model").data))
experiment.dataframe(name="confusion matrix").data

KNeighborsClassifier(n_neighbors=20)


,Adelie,Gentoo,Chinstrap
Adelie,43,0,3
Gentoo,17,1,0
Chinstrap,5,0,35
